In [244]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [245]:
def get_pages(n_page):
    """Devuelve el num de paginas totales

    Args:
        n_page (bs4.element): Soup a escrapear

    Returns:
        int: numero total de paginas
    """
    try:
        
        n_page = soup.find_all('div', class_ = 'haya-pagination mb-2')
        
        n_page = str(n_page)
        
        first_position = int(n_page.find('data-pagination-pages="'))   
        
        last_position = first_position + len('data-pagination-pages="')
        
        pages = n_page[last_position]
        
        for s in n_page[last_position + 1]:
            if s != '"':
                pages = pages + s
    except:
        pages = 1       
        
    return int(pages)


## BUY

In [248]:
#Create lists
links = []
references = []
names = []
prices = []
meters = []
locations = []
rooms = []
baths = []
categories = []

#url
url = 'https://www.haya.es/comprar/viviendas/madrid/'

#Get the request
req = requests.get(url).text

#Create soup
soup = BeautifulSoup(req, 'lxml')

#Get total number of pages
n_pages = get_pages(soup)

#Get houses data
houses = soup.find_all('article', class_='col-12')

#Iterate over pages
for i in range(n_pages):
    
    #Initial url        
    url_init = url
    
    #Iterate over url
    if i != 0:
        
        #Next url
        url_str = url_init + '?p=' + str(i + 1)
        
        #Get request
        req = requests.get(url_str).text
        
        #Create soup
        soup = BeautifulSoup(req, 'lxml')
        #Get houses data
        houses = soup.find_all('article', class_='col-12')
       
    #Iterate over houses    
    for house in houses:
        #Avoid NoNe type error
        if isinstance(house.find('a', class_= 'text-black'),type(None)) == False:
            
            #Get link and append
            try:
                links.append(house.find('a', class_= 'text-black').get('href'))
            except:
                links.append('-')
                
            #Get reference and append
            try:
                references.append(int(house.find('a', class_= 'text-black').get('href').split('-')[1].replace('/','')))
            except:
                references.append('-')
            
            #Get house name and appendd   
            try:
                names.append(house.find('a', class_= 'text-black').get('title'))
            except:
                names.append('-')
                
            #Get location and append
            try:
                locations.append(re.findall(r'\.\s.+\,', house.find('a', class_= 'text-black').get('title'))[0].replace('. ','').replace(',',''))
            except:
                locations.append('-')
            
            #Get category and append
            try:
                categories.append(house.find('a', class_= 'text-black').get('title').split(" ")[0])
            except:
                categories.append('-')
            
            #Get price and append    
            try:
                prices.append(int("".join(re.findall(r'([0-9]+)', house.find('span', class_='text-24 text-lg-28 text-bold ml-auto').text))))
            except:
                prices.append(0)
            
            #Get meters, rooms, baths and append    
            try:    
                num_vars = house.find_all('span', class_= 'text-20 text-medium')
                
                #Get meters
                try:
                    meters.append(int(re.findall(r'([0-9]+)', num_vars[0].text)[0]))
                except:
                    meters.append(0)
                
                #Get rooms
                try:    
                    rooms.append(int(num_vars[1].text.replace(' ', '')))
                except:
                    rooms.append(0)
                
                #Get baths
                try:    
                    baths.append(int(num_vars[2].text.replace(' ','')))
                except:
                    baths.append(0)    
                
            except:
                #Append num vals
                meters.append(0)
                rooms.append(0)
                baths.append(0)
                
        else:
            #Append num vals
            meters.append(0)
            rooms.append(0)
            baths.append(0)
            
            #Get link and append
            try:
                links.append(house.find('a', class_= 'text-white').get('href'))
            except:
                links.append('-')
                
            #Get reference and append
            try:
                references.append(int(house.find('a', class_= 'text-white').get('href').split('-')[1].replace('/','')))
            except:
                references.append('-')    
                
            #Get house name and append    
            try:
                names.append(house.find('a', class_= 'text-white').text) 
            except:
                names.append('-')
            
            #Get location and append
            try:
                locations.append(re.findall(r'\.\s.+\,', house.find('a', class_= 'text-white').text)[0].replace('. ','').replace(',',''))
            except:
                locations.append('-')
                
            #Get category and append
            try:
                categories.append(house.find('a', class_= 'text-white').text)
            except:
                categories.append('-')
            
            #Get price and append    
            try:    
                prices.append(int("".join(re.findall(r'([0-9]+)',house.find('strong', class_='text-24 text-lg-28 text-bold').text))))
            except:
                prices.append(0)

#Print lengths to check results
print(len(links), len(names) , len(prices), len(references), len(categories), len(meters), len(rooms), len(baths))

221 221 221 221 221 221 221 221


In [249]:
#Create rent data frame
df_haya_buy = pd.DataFrame()

#Create columns
df_haya_buy['Referencia'] = references
df_haya_buy['Categoria'] = categories
df_haya_buy['Precio'] = prices
df_haya_buy['Metros'] = meters
df_haya_buy['Habitaciones'] = rooms
df_haya_buy['Baños'] = baths
df_haya_buy['Nombre'] = names
df_haya_buy['Link'] = links

#Print head
print(df_haya_buy.head())

  Referencia  Categoria  Precio  Metros  Habitaciones  Baños  \
0    7238157     Chalet  190000     243             1      1   
1    7259552     Chalet   18000     206             1      1   
2          -  Viviendas  360600      76             2      2   
3    7270726       Piso   12000      61             2      1   
4    7042718       Piso  115000     102             2      2   

                                              Nombre  \
0  Chalet independiente en calle Islas De Cabo Ve...   
1  Chalet adosado en calle Mirador Del Rio. Velil...   
2  Viviendas de 1 habitación en calle Valverde. M...   
3          Piso en calle Marmolistas. Madrid, Madrid   
4  Piso en calle Dos De Mayo. Morata de Tajuña, M...   

                                                Link  
0              https://www.haya.es/vivienda-7238157/  
1              https://www.haya.es/vivienda-7259552/  
2  https://www.haya.es/promocion-conjunta/promoci...  
3              https://www.haya.es/vivienda-7270726/  
4  

In [250]:
#Inspect df
print(df_haya_buy.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221 entries, 0 to 220
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Referencia    221 non-null    object
 1   Categoria     221 non-null    object
 2   Precio        221 non-null    int64 
 3   Metros        221 non-null    int64 
 4   Habitaciones  221 non-null    int64 
 5   Baños         221 non-null    int64 
 6   Nombre        221 non-null    object
 7   Link          221 non-null    object
dtypes: int64(4), object(4)
memory usage: 13.9+ KB
None


In [251]:
#Check unique categories
print(df_haya_buy['Categoria'].value_counts())

#Create standard cat list
cat_list = ['Piso', 'Chalet', 'Viviendas', ' Casa', 'Dúplex', 'Vivienda', 'Pisos']

#Clean categories
df_haya_buy['Categoria'] = df_haya_buy['Categoria'].apply(lambda x: x if x in cat_list else '-')

#Print results
print(df_haya_buy['Categoria'].value_counts())

Piso                     164
Chalet                    39
Viviendas                  4
Casa                       3
Dúplex                     2
Vivienda                   2
Residencial Barquillo      1
MADRE                      1
Garajes,                   1
Pisos                      1
112876-01                  1
147853-02                  1
H13733-01                  1
Name: Categoria, dtype: int64
Piso         164
Chalet        39
-              9
Viviendas      4
Dúplex         2
Vivienda       2
Pisos          1
Name: Categoria, dtype: int64


In [252]:
#Create excel file
excel = df_haya_buy
excel.to_excel(r'C:\Users\Depoi\OneDrive\Escritorio\Excel\Compra_Haya.xlsx', index = False)

## RENT

In [253]:
#Create lists
links = []
references = []
names = []
prices = []
meters = []
locations = []
rooms = []
baths = []
categories = []

#url
url = 'https://www.haya.es/alquiler/viviendas/madrid/'

#Get the request
req = requests.get(url).text

#Create soup
soup = BeautifulSoup(req, 'lxml')

#Get total number of pages
n_pages = get_pages(soup)

#Get houses data
houses = soup.find_all('article', class_='col-12')

#Iterate over pages
for i in range(n_pages):
    
    #Initial url        
    url_init = url
    
    #Iterate over url
    if i != 0:
        
        #Next url
        url_str = url_init + '?p=' + str(i + 1)
        
        #Get request
        req = requests.get(url_str).text
        
        #Create soup
        soup = BeautifulSoup(req, 'lxml')
        #Get houses data
        houses = soup.find_all('article', class_='col-12')
       
    #Iterate over houses    
    for house in houses:
        #Avoid NoNe type error for promotions
        if isinstance(house.find('a', class_= 'text-black'),type(None)) == False:
            
            #Get link and append
            try:
                links.append(house.find('a', class_= 'text-black').get('href'))
            except:
                links.append('-')
                
            #Get reference and append
            try:
                references.append(int(house.find('a', class_= 'text-black').get('href').split('-')[1].replace('/','')))
            except:
                references.append('-')
            
            #Get house name and appendd   
            try:
                names.append(house.find('a', class_= 'text-black').get('title'))
            except:
                names.append('-')
                
            #Get location and append
            try:
                locations.append(re.findall(r'\.\s.+\,', house.find('a', class_= 'text-black').get('title'))[0].replace('. ','').replace(',',''))
            except:
                locations.append('-')
            
            #Get category and append
            try:
                categories.append(house.find('a', class_= 'text-black').get('title').split(" ")[0])
            except:
                categories.append('-')
            
            #Get price and append    
            try:
                prices.append(int("".join(re.findall(r'([0-9]+)', house.find('span', class_='text-24 text-lg-28 text-bold ml-auto').text))))
            except:
                prices.append(0)
            
            #Get meters, rooms, baths and append    
            try:    
                num_vars = house.find_all('span', class_= 'text-20 text-medium')
                
                #Get meters
                try:
                    meters.append(int(re.findall(r'([0-9]+)', num_vars[0].text)[0]))
                except:
                    meters.append(0)
                
                #Get rooms
                try:    
                    rooms.append(int(num_vars[1].text.replace(' ', '')))
                except:
                    rooms.append(0)
                
                #Get baths
                try:    
                    baths.append(int(num_vars[2].text.replace(' ','')))
                except:
                    baths.append(0)    
                
            except:
                #Append num vals
                meters.append(0)
                rooms.append(0)
                baths.append(0)
                
        else:
            #Append num vals
            meters.append(0)
            rooms.append(0)
            baths.append(0)
            
            #Get link and append
            try:
                links.append(house.find('a', class_= 'text-white').get('href'))
            except:
                links.append('-')
                
            #Get reference and append
            try:
                references.append(int(house.find('a', class_= 'text-white').get('href').split('-')[1].replace('/','')))
            except:
                references.append('-')    
                
            #Get house name and append    
            try:
                names.append(house.find('a', class_= 'text-white').text) 
            except:
                names.append('-')
            
            #Get location and append
            try:
                locations.append(re.findall(r'\.\s.+\,', house.find('a', class_= 'text-white').text)[0].replace('. ','').replace(',',''))
            except:
                locations.append('-')
                
            #Get category and append
            try:
                categories.append(house.find('a', class_= 'text-white').text)
            except:
                categories.append('-')
            
            #Get price and append    
            try:    
                prices.append(int("".join(re.findall(r'([0-9]+)',house.find('strong', class_='text-24 text-lg-28 text-bold').text))))
            except:
                prices.append(0)
                
#Print lengths to check results
print(len(links), len(names) , len(prices), len(references), len(categories), len(meters), len(rooms), len(baths))


30 30 30 30 30 30 30 30


In [254]:
#Create rent data frame
df_haya_rent = pd.DataFrame()

#Create columns
df_haya_rent['Referencia'] = references
df_haya_rent['Categoria'] = categories
df_haya_rent['Precio'] = prices
df_haya_rent['Metros'] = meters
df_haya_rent['Habitaciones'] = rooms
df_haya_rent['Baños'] = baths
df_haya_rent['Nombre'] = names
df_haya_rent['Link'] = links

#Print head
print(df_haya_rent.head())

  Referencia     Categoria  Precio  Metros  Habitaciones  Baños  \
0    7053239          Piso    1350     107             3      2   
1    7266265  Estudio/loft     610      26             1      0   
2    7254231          Piso     760      54             3      1   
3    7046954          Piso     720      52             2      1   
4    7050426          Piso     720      57             1      1   

                                              Nombre  \
0  Piso en avenida Dr Federico Rubio Y Gali. Madr...   
1    Estudio/loft en calle Ana Maria. Madrid, Madrid   
2       Piso en calle Carolina Baeza. MADRID, Madrid   
3    Piso en calle Agustina Aragon. Móstoles, Madrid   
4                Piso en calle Ansar. Madrid, Madrid   

                                    Link  
0  https://www.haya.es/vivienda-7053239/  
1  https://www.haya.es/vivienda-7266265/  
2  https://www.haya.es/vivienda-7254231/  
3  https://www.haya.es/vivienda-7046954/  
4  https://www.haya.es/vivienda-7050426/  


In [255]:
#Inspect df
print(df_haya_rent.info())

#Check unique categories
print(df_haya_rent['Categoria'].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Referencia    30 non-null     object
 1   Categoria     30 non-null     object
 2   Precio        30 non-null     int64 
 3   Metros        30 non-null     int64 
 4   Habitaciones  30 non-null     int64 
 5   Baños         30 non-null     int64 
 6   Nombre        30 non-null     object
 7   Link          30 non-null     object
dtypes: int64(4), object(4)
memory usage: 2.0+ KB
None
Piso                                         25
Estudio/loft                                  1
Pisos de 2 y 3 dormitorios desde 895€/mes     1
                                              1
Viviendas de alquiler en C/ Pedro Faura       1
Pisos alquiler alta suministros gratis!       1
Name: Categoria, dtype: int64


In [256]:
#Check unique categories
print(df_haya_rent['Categoria'].value_counts())

#Create standard cat list
cat_list = ['Piso', 'Chalet', 'Viviendas', ' Casa', 'Dúplex', 'Vivienda', 'Pisos', 'Estudio/loft']

#Clean categories
df_haya_rent['Categoria'] = df_haya_rent['Categoria'].apply(lambda x: x if x in cat_list else '-')

#Print results
print(df_haya_rent['Categoria'].value_counts())

Piso                                         25
Estudio/loft                                  1
Pisos de 2 y 3 dormitorios desde 895€/mes     1
                                              1
Viviendas de alquiler en C/ Pedro Faura       1
Pisos alquiler alta suministros gratis!       1
Name: Categoria, dtype: int64
Piso            25
-                4
Estudio/loft     1
Name: Categoria, dtype: int64


In [257]:
#Create excel file
excel = df_haya_rent
excel.to_excel(r'C:\Users\Depoi\OneDrive\Escritorio\Excel\Alquiler_Haya.xlsx', index = False)